# Administration of Glowing Bear data warehouse with the Python client
---------------

Interacting with the data in the [tranSMART Glowing Bear data warehouse](https://glowingbear.app) via the [REST API](https://glowingbear.app/docs/technical/#transmart-api) and the [Python client](https://github.com/thehyve/transmart-api-client-py) into the Jupyter Notebook analytical environment.

Note that you need admin rights on the tranSMART Glowing Bear server to execute the below calls.

In [4]:
import transmart as tm
print('transmart python client version: {}'.format(tm.__version__))

transmart python client version: 0.2.4


Generate an offline token for your user:

In [5]:
import requests

# Keycloak credentials you also use to access Glowing Bear
user = None
password = None
kc_token_url = "https://keycloak-dwh-test.thehyve.net/auth/realms/transmart/protocol/openid-connect/token"
kc_client_id= "transmart-client"

r = requests.post(url=kc_token_url,
                  data=dict(grant_type='password',
                            client_id=kc_client_id,
                            scope= 'offline_access',
                            username=user,
                            password=password
                           )
                 )
offline_token = r.json().get('refresh_token')
offline_token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJKREJWbTE4TWM3V181d2V0WlI1OUN1RU5WMzd1dEF1VXdsNm9mcjlscmU0In0.eyJqdGkiOiIwYThkZWU1Ni1jMzM3LTRmNWItODI0Zi0xOThiYWFlYTM5NTYiLCJleHAiOjAsIm5iZiI6MCwiaWF0IjoxNTYyMTQwMjMyLCJpc3MiOiJodHRwczovL2tleWNsb2FrLWR3aC10ZXN0LnRoZWh5dmUubmV0L2F1dGgvcmVhbG1zL3RyYW5zbWFydCIsImF1ZCI6InRyYW5zbWFydC1jbGllbnQiLCJzdWIiOiI3YjU2MGI5NS1iNjM0LTQ5YzYtYjRkYi1kMWM2MjhmOWQzNjMiLCJ0eXAiOiJPZmZsaW5lIiwiYXpwIjoidHJhbnNtYXJ0LWNsaWVudCIsImF1dGhfdGltZSI6MCwic2Vzc2lvbl9zdGF0ZSI6ImE2NjAxODdhLTk4ZGYtNGUyMS05OTVmLWJkMTk5OGM1Yjc5YiIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fX0.R7280jvdyWaQGmAWa0uNsnCBIDtFUuNQbijJHqst6Mu-R9SFi7c3h3rd_0ZqTVHNWXuNBS65uTaOTnI3H7PZOhNC5fg4kY6v8SXgb3u5Rwd3Ug1A23oevXOoaYLwbXxH1QdtA1XH9aCp5C89rHa3dBeVr7gFp_hc7QggnBkGt5i0cW47NZR_vjHtBEunfbiD90Gfgg3G-LKMeaqhNZytJAPYIpaSb0NIwHWyTJWGyiB

Connect to the tranSMART Glowing Bear server:

In [6]:
api = tm.get_api(
    host = 'https://transmart.thehyve.net', # URL of tranSMART server connected to your Glowing Bear
    kc_url = "https://keycloak-dwh-test.thehyve.net", # URL of Keycloak connected to your Glowing Bear
    kc_realm = "transmart", # Realm in Keycloak for the tranSMART application
    offline_token=offline_token,
    print_urls = True # Whether or not to print the API URLs used behind the scenes
)

# Common errors:
# * '401 Client Error: Unauthorized' - Wrong username/password
# * 'HTTPSConnectionPool' - Wrong tranSMART or Keycloak URL or no internet
# * '404 Client Error: Not Found' - Wrong Keycloak realm

https://transmart.thehyve.net/v2/studies
https://transmart.thehyve.net/v2/tree_nodes?depth=0&counts=False&tags=True
Existing index cache found. Loaded 10263 tree nodes. Hooray!
https://transmart.thehyve.net/v2/pedigree/relation_types


After you are connected and have loaded the new data, execute the following call to rebuild the necessary caches and see your changes in Glowing Bear:

In [7]:
# Clear cache
api.admin.after_data_loading_update()

https://transmart.thehyve.net/v2/admin/system/after_data_loading_update


{'timestamp': 1562140234912,
 'status': 403,
 'error': 'Forbidden',
 'message': 'Access is denied',
 'path': '/v2/admin/system/after_data_loading_update'}

The following calls are examples of sanity checks on whether the data loading has succeeded:

In [8]:
# Check if your new dataset / study is there
api.get_studies().dataframe

https://transmart.thehyve.net/v2/studies


,bioExperimentId,dimensions,id,secureObjectToken,studyId
0,None,"[study, patient, start time, concept]",2,PUBLIC,SYNTHETICMASS
1,None,"[study, Images Id, patient, concept]",4,PUBLIC,IMAGES
2,None,"[study, Biomaterial ID, patient, concept, Diag...",7,PUBLIC,Tumor Samples


In [9]:
# Check if your new part of the tree is there
tree = api.tree_nodes(root='\\', counts=True, depth=2)
tree

https://transmart.thehyve.net/v2/tree_nodes?root=\&depth=2&counts=True&tags=True


Conditions  (None)/
  ICD10  (None)
  SNOMED  (None)
Demographics  (None)/
  Birthdate  (1462)
  Birthplace  (1462)
  Deathdate  (457)
  Ethnicity  (1462)
  Gender  (1462)
  Marital  (1033)
  Race  (1462)
Public Studies  (None)/
  IMAGES  (5)
  Tumor Samples  (9)